In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from keras_tqdm import TQDMNotebookCallback
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tqdm import tqdm_notebook

from fastmri_recon.data.fastmri_tf_datasets import train_masked_kspace_dataset
from fastmri_recon.models.cascading import cascade_net

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

n_samples_train = 7135
n_samples_val = 7135

n_volumes_train = 199
n_volumes_val = 199

In [4]:
# generators
AF = 4
train_ds = train_masked_kspace_dataset(train_path, AF=AF, inner_slices=8, rand=True, scale_factor=1e6)
val_ds = train_masked_kspace_dataset(val_path, AF=AF, scale_factor=1e6)

In [5]:
run_params = {
    'n_cascade': 5, 
    'n_convs': 5, 
    'n_filters': 48,
    'noiseless': True,
}

n_epochs = 10
run_id = f'cascadenet_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

cascadenet_af4_1575192493


In [6]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=100, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
    profile_batch=0,
)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
tqdm_cb.on_train_batch_begin = tqdm_cb.on_batch_begin
tqdm_cb.on_train_batch_end = tqdm_cb.on_batch_end

W1201 10:28:13.983943 140515613492992 callbacks.py:863] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [7]:
model = cascade_net(lr=1e-3, **run_params)
print(model.summary(line_length=100))

Model: "model"
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
kspace_input (InputLayer)        [(None, 640, None, 1) 0                                            
____________________________________________________________________________________________________
ifft_simple (Lambda)             (None, 640, None, 1)  0           kspace_input[0][0]               
____________________________________________________________________________________________________
lambda (Lambda)                  (None, 640, None, 1)  0           ifft_simple[0][0]                
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 640, None, 1)  0           ifft_simple[0][0]                
____________________________________________________________________________

In [8]:
%%time
# simple overfit trials
data = next(iter(train_ds))
model.fit(
    x=data[0], 
    y=data[1], 
    batch_size=data[0][0].shape[0], 
    epochs=500,
    verbose=2, 
    shuffle=False,
)

Train on 1 samples
Epoch 1/500
1/1 - 6s - loss: 11.2858 - mean_squared_error: 239.7561 - keras_psnr: 27.0884 - keras_ssim: 0.6466
Epoch 2/500
1/1 - 0s - loss: 10.8140 - mean_squared_error: 214.0250 - keras_psnr: 27.5814 - keras_ssim: 0.6568
Epoch 3/500
1/1 - 0s - loss: 10.3634 - mean_squared_error: 189.5505 - keras_psnr: 28.1088 - keras_ssim: 0.6654
Epoch 4/500
1/1 - 0s - loss: 10.0225 - mean_squared_error: 172.9082 - keras_psnr: 28.5079 - keras_ssim: 0.6712
Epoch 5/500
1/1 - 0s - loss: 9.9896 - mean_squared_error: 171.1943 - keras_psnr: 28.5512 - keras_ssim: 0.6729
Epoch 6/500
1/1 - 0s - loss: 9.8765 - mean_squared_error: 166.3442 - keras_psnr: 28.6760 - keras_ssim: 0.6763
Epoch 7/500
1/1 - 0s - loss: 9.7668 - mean_squared_error: 162.5140 - keras_psnr: 28.7772 - keras_ssim: 0.6786
Epoch 8/500
1/1 - 0s - loss: 9.6760 - mean_squared_error: 159.1117 - keras_psnr: 28.8691 - keras_ssim: 0.6806
Epoch 9/500
1/1 - 0s - loss: 9.6097 - mean_squared_error: 156.9427 - keras_psnr: 28.9287 - keras_

KeyboardInterrupt: 